In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

1.	**Brand** - Brand and make of the vehicle.
2.	**Type** - Model type.
3.	**Reg_date** - Registration date of the vehicle.
4.	**Coe_left** - Balance lifespan on the road.
5.	**Dep** - Annual Depreciation.
6.	**Mileage** - Total Mileage clocked on the vehicle.
7.	**Road Tax** - Annual tax payable for public road usage.
8.	**Dereg Value** - The amount rebate if the vehicle is scrapped or exported today(May 2021)
9.	**COE (certificate of entitlement)** - Price of COE paid.
10.	**Engine Cap** - Engine capacity in CC (cubic capacity).
11. **OMV** - Open Market Value (specifical to Singapore).
12.	**ARF** - Additional Registration Fee.
13.	**Power**.
14.	**No. of Owners**.
15.	**Price**.

In [2]:
# Load the data
data = pd.read_csv('EXAM_5\data.csv')
data = data.drop('Unnamed: 18', axis=1)

In [3]:
#Drop the missing target values
data = data.drop(data[data['Price'] == 'N.A'].index.tolist())

Exploring where are missing values

In [4]:
data

,Brand,Type,Reg_date,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price
0,BMW 5 Series 520i Highline,Luxury Sedan,28-May-13,1yr 11mths 21days COE left),15920,N.A,1210,38133,67304,1997,1610,2013,Auto,41566,45193,135,1,54000
1,Honda Vezel 1.5A X Honda Sensing,SUV,14-Oct-15,4yrs 4mths 7days COE left),10310,50000,682,31237,56001,1496,1190,2015,Auto,19775,9775,96,2,49800
2,MINI One 1.6A (New 10-yr COE),Hatchback,7-Oct-11,10yrs COE left),6280,110000,742,N.A,N.A,1598,1100,2011,Auto,18180,18180,72,3,62800
4,Suzuki Swift Sport 1.6M (COE till 12/2022),Hatchback,28-Dec-07,1yr 6mths 21days COE left),8210,203000,1030,6656,21349,1586,1060,2007,Manual,12154,13370,92,2,12800
5,Porsche 718 Cayman 2.0A PDK,Sports Car,19-Jul-17,6yrs 1mth 12days COE left),34200,21000,1200,106829,50110,1988,1365,2017,Auto,71979,101563,220,1,259988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,Mercedes-Benz C-Class C180,Luxury Sedan,20-Dec-13,2yrs 6mths 13days COE left),15240,127000,740,35772,73989,1595,1485,2013,Auto,29525,28335,115,4,52828
4407,Mazda 3 1.5A Sunroof,Mid-Sized Sedan,7-Dec-17,6yrs 6mths COE left),9340,40200,682,35419,42801,1496,1310,2017,Auto,15108,10108,88,3,65800
4408,Toyota Hiace 2.5A (COE till 02/2022),Bus/Mini Bus,8-Feb-07,8mths 1day COE left),26410,N.A,N.A,3269,24252,2494,1800,2006,Auto,N.A,27970,6,N.A,17800
4409,Subaru Forester 2.0i-L Sunroof,SUV,29-Jul-15,4yrs 1mth 22days COE left),10590,120000,1208,39265,74501,1995,1566,2015,Auto,13772,13772,110,1,50800


In [5]:
#replace N.A and N.A. with None value
for column in data.columns:
    data.loc[(data[column] == 'N.A.') | (data[column] == 'N.A'), column] = np.NAN

In [6]:
data.isnull().sum()

Brand              0
Type               0
Reg_date           6
Coe_left           6
Dep               51
Mileage          846
Road Tax         396
Dereg Value      242
COE              230
Engine Cap         3
Curb Weight       60
Manufactured       5
Transmission       0
OMV              388
ARF                3
Power              7
No. of Owners     49
Price              0
dtype: int64

Since none of the columns have more than 60% missing values, we won't drop them

In [7]:
# Dealing with Registration Date
data = data.dropna(subset=['Reg_date'])
data = data.dropna(subset=['Coe_left'])

def date(li):
    return int(li.split('-')[-1])

data['Reg_date'] = data['Reg_date'].map(date)

In [8]:
# Dealing with Coe_left and other categorical data
data8 = data['Coe_left'].str.split('COE left', expand = True)
data9 = data8[0].str.split(expand=True)

data9[0] = data9[0].str.split('y')
data9[1] = data9[1].str.split('m')
data9[2] = data9[2].str.split('d')

data9.fillna('0', inplace=True)

def firstEl(li):
    if li[0].isdigit():
        return li[0]
    
data9['years'] = data9[0].map(firstEl)
data9['months'] = data9[1].map(firstEl)
data9['days'] = data9[2].map(firstEl)

data9.fillna('0', inplace=True)

newdata = data9.drop([0, 1, 2], axis=1)

# Take the input and return an int
def digitalize(inp):
    return int(inp)

data9['years'] = data9['years'].map(digitalize)
data9['months'] = data9['months'].map(digitalize)
data9['days'] = data9['days'].map(digitalize)

data9['final'] = data9.years + data9.months / 12 + data9.days / 365
data['Coe_left'] = data9['final']

data10 = data['Brand'].str.split(expand=True)
data10['brand'] = data10[0]
data['Brand'] = data10['brand']

In [9]:
#Encoding the categorical data: Transmission, Brand, Type
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()

data['Transmission'] = encoder.fit_transform(np.array(data['Transmission']).reshape(-1, 1))
data['Brand'] = encoder.fit_transform(np.array(data['Brand']).reshape(-1, 1))
data['Type'] = encoder.fit_transform(np.array(data['Type']).reshape(-1, 1))

In [10]:
missing_columns = ['Mileage', 'Road Tax', 'Dereg Value', 'COE', 'OMV']

In [14]:
#Missing data imputation using regression
from sklearn.linear_model import LinearRegression

parameters = list(set(data.columns) - set(missing_columns))

#The parameters need to be clean, since they've fewer than 100 missing instances, replace NaN with median
for column in parameters:
    data[column].fillna(data[column].median(), inplace=True)
    
model = LinearRegression()
#For the target columns, replace NaN with prediction based on parameters
for feature in missing_columns:
    model.fit(X = data.dropna(subset=[feature])[parameters], y = data.dropna(subset=[feature])[feature])
    data.loc[data[feature].isnull(), feature] = model.predict(data[parameters])[data[feature].isnull()]

In [15]:
data.isnull().sum()

Brand            0
Type             0
Reg_date         0
Coe_left         0
Dep              0
Mileage          0
Road Tax         0
Dereg Value      0
COE              0
Engine Cap       0
Curb Weight      0
Manufactured     0
Transmission     0
OMV              0
ARF              0
Power            0
No. of Owners    0
Price            0
dtype: int64

In [16]:
data

,Brand,Type,Reg_date,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price
0,3.0,2.0,13,1.974201,15920,100132.57059,1210,38133,67304,1997,1610,2013,0.0,41566,45193,135,1,54000
1,14.0,6.0,15,4.352511,10310,50000,682,31237,56001,1496,1190,2015,0.0,19775,9775,96,2,49800
2,27.0,1.0,11,10.000000,6280,110000,742,23173.69197,28731.123228,1598,1100,2011,0.0,18180,18180,72,3,62800
4,48.0,1.0,7,1.557534,8210,203000,1030,6656,21349,1586,1060,2007,2.0,12154,13370,92,2,12800
5,38.0,7.0,17,6.116210,34200,21000,1200,106829,50110,1988,1365,2017,0.0,71979,101563,220,1,259988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,32.0,2.0,13,2.535616,15240,127000,740,35772,73989,1595,1485,2013,0.0,29525,28335,115,4,52828
4407,30.0,4.0,17,6.500000,9340,40200,682,35419,42801,1496,1310,2017,0.0,15108,10108,88,3,65800
4408,49.0,0.0,7,0.000000,26410,175771.82557,2171.028272,3269,24252,2494,1800,2006,0.0,23691.271372,27970,6,2.0,17800
4409,47.0,6.0,15,4.143607,10590,120000,1208,39265,74501,1995,1566,2015,0.0,13772,13772,110,1,50800


In [17]:
parameters

['Manufactured',
 'Power',
 'Coe_left',
 'No. of Owners',
 'Dep',
 'Engine Cap',
 'Curb Weight',
 'Transmission',
 'ARF',
 'Type',
 'Price',
 'Brand',
 'Reg_date']

In [18]:
for index, column in enumerate(data.columns):
    print(f'{index} : {column}')

0 : Brand
1 : Type
2 : Reg_date
3 : Coe_left
4 : Dep
5 : Mileage
6 : Road Tax
7 : Dereg Value
8 : COE
9 : Engine Cap
10 : Curb Weight
11 : Manufactured
12 : Transmission
13 : OMV
14 : ARF
15 : Power
16 : No. of Owners
17 : Price


In [19]:
data.to_csv('data_ready.csv', index=False)